### **Aluno: Leonardo Ribeiro Schaedler**

# Atividade MapReduce

Considerando o dataset detalhado a seguir, extraia o conjunto de informações solicitadas.

### Dataset dados de operacoes comerciais entre paises
- Dados relativos a transacoes comerciais entre paises ao longo do tempo
- 2 bases:
  - base.csv ~433MB (3M de eventos)
  - base_sample.csv ~4MB (35k de eventos)
- 4.4M de instâncias

### Formato da base

|  # | Nome do campo        	| Descrição                                     	|
|----|----------------------	|-----------------------------------------------	|
|  0 | 	País	País envolvido na transação comercial|
|  1 | 	Ano	Ano em que a transação foi efetuada|
|  2 | 	Código	Código da mercadoria|
|  3 | 	Mercadoria 	Descrição da mercadoria|
|  4 | 	Fluxo	Fluxo, e.g. Exportação ou Importação|
|  5 | 	Valor	Valor em dólares|
|  6 | 	Peso	Peso da mercadoria|
|  7 | 	Unidade 	Unidade de medida da mercadoria, e.g. Quantidade de itens|
|  8 | 	Quantidade	Quantidade conforme a unidade especificada da mercadoria|
|  9 | 	Categoria	Categoria da mercadoria, e.g. Produto Animal|


### Informações a serem extraídas:

1.	País com a maior quantidade de transações comerciais efetuadas;
2.	Mercadoria com a maior quantidade de transações comerciais no Brasil (como a base de dados está em inglês utilize Brazil, com Z, utilize a função “contains” do Java);
3.	Quantidade de transações financeiras realizadas por ano;
4.	Mercadoria com maior quantidade de transações financeiras;
5.	Mercadoria com maior quantidade de transações financeiras em 2016;
6.	Mercadoria com maior quantidade de transações financeiras em 2016, no Brasil (como a base de dados está em inglês utilize Brazil, com Z);
7.	Mercadoria com maior total de peso, de acordo com todas transações comerciais;
8.	Mercadoria com maior total de peso, de acordo com todas transações comerciais, separadas de acordo com o ano;
9.	Média de peso por mercadoria, separadas de acordo com o ano;
10.	Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), separadas de acordo com o ano;
11.	Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), em relação ao fluxo, separadas de acordo com o ano;
12.	Preco medio das commodities de acordo com o ano
13.	Valores maximos, minimos e médios de cada tipo de mercadoria por ano
14.	Pais com o maior preco da commodity em fluxos do tipo Export
15.	Quantidade de transações comerciais de acordo com o fluxo, de acordo com o ano;

In [166]:
#Instala ambiente local do MapReduce
!pip --quiet install mrjob
#Efetua download da base completa (400MB)  https://drive.google.com/file/d/1LdeDR5wKP15kUywI7491OmKzdMzVgO-c/view?usp=sharing
!gdown 1LdeDR5wKP15kUywI7491OmKzdMzVgO-c
#Efetua download da base parcial (4MB)   https://drive.google.com/file/d/1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm/view?usp=sharing
!gdown 1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm

Downloading...
From: https://drive.google.com/uc?id=1LdeDR5wKP15kUywI7491OmKzdMzVgO-c
To: /content/base.csv
100% 444M/444M [00:02<00:00, 214MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm
To: /content/base_sample.csv
100% 4.06M/4.06M [00:00<00:00, 166MB/s]


### Informação 1
 - País com a maior quantidade de transações comerciais efetuadas;

In [167]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        pais = campos[0]
        yield(pais, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + 1
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial (Top 1 = "Canada"	936)

In [168]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223514.725466
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223514.725466...

Top 10 Entradas da atividade
"Canada"	936
"France"	762
"Germany"	740
"Netherlands"	717
"Greece"	687
"Italy"	655
"Denmark"	646
"Czech Rep."	643
"Malaysia"	640
"Austria"	614


Teste com base completa (Top 1 = "Australia"	89487)

In [169]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223516.307517
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223516.307517...

Top 10 Entradas da atividade
"Australia"	89487
"Canada"	69468
"Austria"	58683
"Argentina"	57729
"China, Hong Kong SAR"	55535
"Brazil"	54356
"Denmark"	49905
"Belgium"	49525
"China"	49275
"France"	46052


### Informação 2
 - Mercadoria com a maior quantidade de transações comerciais no Brasil (como a base de dados está em inglês utilize Brazil, com Z)

In [170]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        country = campos[0]
        if country == "Brazil":
          comm = campos[3]
          yield(comm, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [171]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223602.657764
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223602.657764...

Top 10 Entradas da atividade
"Horses, live except pure-bred breeding"	64
"Animals, live, except farm animals"	56
"Bovine animals, live pure-bred breeding"	56
"Fowls, live domestic < 185 grams"	56
"Bovine animals, live, except pure-bred breeding"	54
"Horses, live pure-bred breeding"	54
"Swine, live pure-bred breeding"	52
"Sheep, live"	47
"Poultry, live except domestic fowls, < 185 grams"	32
"Goats, live"	30


Teste com base completa

In [172]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223603.302072
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223603.302072...

Top 10 Entradas da atividade
"Industrial fatty alcohols"	84
"Cigarette or pipe tobacco and tobacco substitute mixe"	69
"Synthetic organic pigments and preps based thereon"	69
"Beauty, makeup and suntan preparations nes"	68
"Chocolate/cocoa food preparations nes"	68
"Chocolate, cocoa preps, block, slab, bar, filled, >2k"	68
"Essential oils, nes"	68
"Hair preparations, nes"	68
"Hair shampoos"	68
"Jams, fruit jellies, purees and pastes, except citrus"	68


### Informação 3
 - Quantidade de transações financeiras realizadas por ano;

In [173]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        yield(year, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [174]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223613.784920
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223613.784920...

Top 10 Entradas da atividade
"2012"	1628
"2013"	1621
"2003"	1580
"2010"	1571
"2014"	1568
"2011"	1566
"2001"	1557
"2015"	1551
"2000"	1549
"2007"	1538


Teste com base completa

In [175]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223614.811797
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223614.811797...

Top 10 Entradas da atividade
"2012"	137457
"2011"	137418
"2010"	137198
"2009"	136645
"2006"	135996
"2007"	135855
"2013"	135501
"2005"	133867
"2014"	133270
"2008"	133210


### Informação 4
 - Mercadoria com maior quantidade de transações financeiras;

In [176]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        comm = campos[3]
        yield(comm, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [177]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223654.035698
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223654.035698...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	4413
"Fowls, live domestic < 185 grams"	3383
"Horses, live except pure-bred breeding"	3331
"Horses, live pure-bred breeding"	2971
"Bovine animals, live, except pure-bred breeding"	2758
"Poultry, live except domestic fowls, < 185 grams"	2494
"Bovine animals, live pure-bred breeding"	2490
"Sheep, live"	2287
"Goats, live"	2002
"Poultry, live except domestic fowls, > 185 grams"	1991


Teste com base completa

In [178]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223655.067155
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223655.067155...

Top 10 Entradas da atividade
"Food preparations nes"	8048
"Sugar confectionery not chewing gum, no cocoa content"	7764
"Sauces nes, mixed condiments, mixed seasoning"	7693
"Cigarettes containing tobacco"	7620
"Communion wafers, rice paper, bakers wares nes"	7530
"Sweet biscuits, waffles and wafers"	7402
"Chocolate/cocoa food preparations nes"	7267
"Refined sugar, in solid form, nes, pure sucrose"	7258
"Animal feed preparations nes"	7242
"Coffee extracts, essences, concentrates, preparations"	7043


### Informação 5
 - Mercadoria com maior quantidade de transações financeiras em 2016

In [179]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        if year == "2016":
          comm = campos[3]
          yield(comm, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [180]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223737.407964
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223737.407964...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	161
"Horses, live except pure-bred breeding"	121
"Fowls, live domestic < 185 grams"	119
"Horses, live pure-bred breeding"	107
"Bovine animals, live, except pure-bred breeding"	98
"Bovine animals, live pure-bred breeding"	89
"Sheep, live"	86
"Swine, live pure-bred breeding"	71
"Goats, live"	70
"Poultry, live except domestic fowls, > 185 grams"	69


Teste com base completa

In [181]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223738.145313
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223738.145313...

Top 10 Entradas da atividade
"Food preparations nes"	278
"Cigarettes containing tobacco"	272
"Communion wafers, rice paper, bakers wares nes"	269
"Sauces nes, mixed condiments, mixed seasoning"	268
"Sugar confectionery not chewing gum, no cocoa content"	267
"Sweet biscuits, waffles and wafers"	267
"Chocolate/cocoa food preparations nes"	258
"Malt extract & limited cocoa pastrycooks products nes"	251
"Animal feed preparations nes"	249
"Coffee extracts, essences, concentrates, preparations"	249


### Informação 6
 - Mercadoria com maior quantidade de transações financeiras em 2016, no Brasil (como a base de dados está em inglês utilize Brazil, com Z);

In [182]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        country = campos[0]
        year = campos[1]
        if country == "Brazil" and year == "2016":
          comm = campos[3]
          yield(comm, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [183]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223748.435233
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223748.435233...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	2
"Bovine animals, live, except pure-bred breeding"	2
"Bovine animals, live pure-bred breeding"	2
"Fowls, live domestic < 185 grams"	2
"Horses, live except pure-bred breeding"	2
"Horses, live pure-bred breeding"	2
"Swine, live pure-bred breeding"	2
"Fowls, live domestic > 185 grams"	1
"Sheep, live"	1
"Swine, live except pure-bred breeding < 50 kg"	1


Teste com base completa

In [184]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223749.185031
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223749.185031...

Top 10 Entradas da atividade
"Acid and mordant dyes and preparations based thereon"	2
"Acrylic or vinyl polymer paint or varnish, non-aqueou"	2
"Acrylic & vinyl polymer based paint, varnish, in wate"	2
"Adrenal cortical hormones, in dosage"	2
"Agar-agar"	2
"Agarbatti, odorifers operated by burning"	2
"Agglomerated dolomite (including tarred dolomite)"	2
"Alcoholic liqueurs nes"	2
"Alkali metals other than sodium"	2
"Alkaloids, derivs, no antibiotics, hormones, in dosag"	2


### Informação 7
 - Mercadoria com maior total de peso, de acordo com todas transações comerciais;

In [185]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        comm = campos[3]
        weight = float(campos[6])
        yield(comm, weight)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [186]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223758.566514
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223758.566514...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding"	72350442141.0
"Swine, live except pure-bred breeding > 50 kg"	38798669146.0
"Fowls, live domestic > 185 grams"	18665789461.0
"Sheep, live"	13771569405.0
"Swine, live except pure-bred breeding < 50 kg"	12228873816.0
"Bovine animals, live pure-bred breeding"	7733896254.0
"Poultry, live except domestic fowls, > 185 grams"	4546972430.0
"Horses, live except pure-bred breeding"	2396204313.0
"Animals, live, except farm animals"	1469485365.0
"Fowls, live domestic < 185 grams"	1404224958.0


Teste com base completa

In [187]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223759.600447
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223759.600447...

Top 10 Entradas da atividade
"Petroleum oils, oils from bituminous minerals, crude"	46002412921988.0
"Iron ore, concentrate, not iron pyrites,unagglomerate"	34878419167261.0
"Ice, snow and potable water not sweetened or flavoure"	25759966772196.0
"Bituminous coal, not agglomerated"	21959118134095.0
"Oils petroleum, bituminous, distillates, except crude"	19216117424056.0
"Natural gas in gaseous state"	10696575625293.0
"Iron ore, concentrate, not iron pyrites, agglomerated"	7116880502870.0
"Coal except anthracite or bituminous, not agglomerate"	6161036788891.0
"Wheat except durum wheat, and meslin"	5452684784869.0
"Natural gas, liquefied"	5297769728104.0


### Informação 8
 - Mercadoria com maior total de peso, de acordo com todas transações comerciais, separadas de acordo com o ano;

In [188]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        comm = campos[3]
        weight = float(campos[6])
        yield(comm + " " + year, weight)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [189]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223844.552222
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223844.552222...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding 2011"	3790561468.0
"Bovine animals, live, except pure-bred breeding 2010"	3725385865.0
"Bovine animals, live, except pure-bred breeding 2009"	3667337654.0
"Bovine animals, live, except pure-bred breeding 2007"	3582928269.0
"Bovine animals, live, except pure-bred breeding 2012"	3532005526.0
"Bovine animals, live, except pure-bred breeding 2006"	3474206442.0
"Bovine animals, live, except pure-bred breeding 2008"	3438256284.0
"Bovine animals, live, except pure-bred breeding 2013"	3438146811.0
"Bovine animals, live, except pure-bred breeding 2015"	3430878298.0
"Bovine animals, live, except pure-bred breeding 2005"	3358493111.0


Teste com base completa

In [190]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223845.686747
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223845.686747...

Top 10 Entradas da atividade
"Iron ore, concentrate, not iron pyrites,unagglomerate 2016"	2706312145364.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2015"	2649415129890.0
"Petroleum oils, oils from bituminous minerals, crude 2010"	2613316703587.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2014"	2555725226600.0
"Petroleum oils, oils from bituminous minerals, crude 2011"	2547369237511.0
"Petroleum oils, oils from bituminous minerals, crude 2014"	2389423152614.0
"Petroleum oils, oils from bituminous minerals, crude 2015"	2334398373730.0
"Petroleum oils, oils from bituminous minerals, crude 2012"	2319929864134.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2013"	2300741188739.

### Informação 9
 - Média de peso por mercadoria, separadas de acordo com o ano;

In [191]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        comm = campos[3]
        weight = float(campos[6])
        yield(comm + " " + year, weight)
      except:
        pass

    def reducer(self, chave, valores):
      count = 0
      valor = 0
      for v in valores:
        valor = valor + v
        count += 1
      avg = valor / count
      yield(chave, avg)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [192]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223932.973329
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223932.973329...

Top 10 Entradas da atividade
"Sheep, live 1988"	54003615.0
"Swine, live except pure-bred breeding > 50 kg 2008"	43181538.152542375
"Swine, live except pure-bred breeding > 50 kg 2011"	36794547.88888889
"Swine, live except pure-bred breeding > 50 kg 2013"	35665136.307692304
"Swine, live except pure-bred breeding > 50 kg 1988"	35478237.5
"Swine, live except pure-bred breeding > 50 kg 2014"	34317240.484375
"Fowls, live domestic > 185 grams 2016"	33725574.385964915
"Swine, live except pure-bred breeding > 50 kg 2007"	32519872.38596491
"Swine, live except pure-bred breeding > 50 kg 2016"	32402963.825396825
"Swine, live except pure-bred breeding > 50 kg 2009"	32227585.830985915


Teste com base completa

In [193]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.223934.000703
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.223934.000703...

Top 10 Entradas da atividade
"Bituminous coal, not agglomerated 1988"	34557636811.333336
"Oils petroleum, bituminous, distillates, except crude 1999"	32343556715.881355
"Petroleum oils, oils from bituminous minerals, crude 1988"	30595841803.444443
"Petroleum oils, oils from bituminous minerals, crude 1991"	20943228268.56
"Petroleum oils, oils from bituminous minerals, crude 1989"	20281057983.18182
"Petroleum oils, oils from bituminous minerals, crude 2010"	19648997771.330826
"Petroleum oils, oils from bituminous minerals, crude 1990"	18362963582.692307
"Petroleum oils, oils from bituminous minerals, crude 2014"	18239871394.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 1988"	18002319954.4375
"Pet

### Informação 10
 - Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), separadas de acordo com o ano;

In [194]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        country = campos[0]
        year = campos[1]
        comm = campos[3]
        weight = float(campos[6])
        if country == "Brazil":
          yield(comm + " " + year, weight)
      except:
        pass

    def reducer(self, chave, valores):
      count = 0
      valor = 0
      for v in valores:
        valor = valor + v
        count += 1
      avg = valor / count
      yield(chave, avg)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [195]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224022.028394
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224022.028394...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding 2010"	163259108.0
"Bovine animals, live, except pure-bred breeding 2013"	161980315.0
"Bovine animals, live, except pure-bred breeding 2014"	151681448.5
"Bovine animals, live, except pure-bred breeding 2009"	134964128.0
"Bovine animals, live, except pure-bred breeding 2012"	121096675.0
"Bovine animals, live, except pure-bred breeding 2008"	105512865.5
"Bovine animals, live, except pure-bred breeding 2007"	102115793.5
"Bovine animals, live, except pure-bred breeding 2011"	98217470.0
"Bovine animals, live, except pure-bred breeding 1995"	63394422.0
"Bovine animals, live, except pure-bred breeding 1994"	52523980.0


Teste com base completa

In [196]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224023.050799
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224023.050799...

Top 10 Entradas da atividade
"Iron ore, concentrate, not iron pyrites,unagglomerate 2016"	172274027120.5
"Iron ore, concentrate, not iron pyrites,unagglomerate 2015"	157510314215.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2014"	147233395393.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2013"	141076354479.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2012"	137699547522.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2011"	137398453341.5
"Iron ore, concentrate, not iron pyrites,unagglomerate 2008"	115846272058.5
"Iron ore, concentrate, not iron pyrites,unagglomerate 2007"	109698561880.5
"Iron ore, concentrate, not iron pyrites,unagglomerate 2006"	98437833303.5
"Ir

### Informação 11
 - Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), em relação ao fluxo, separadas de acordo com o ano;

In [197]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        country = campos[0]
        year = campos[1]
        comm = campos[3]
        flow = campos[4]
        weight = float(campos[6])
        if country == "Brazil":
          yield(comm + " " + year + " " + flow, weight)
      except:
        pass

    def reducer(self, chave, valores):
      count = 0
      valor = 0
      for v in valores:
        valor = valor + v
        count += 1
      avg = valor / count
      yield(chave, avg)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [198]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224032.339557
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224032.339557...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding 2013 Export"	323951180.0
"Bovine animals, live, except pure-bred breeding 2010 Export"	323732156.0
"Bovine animals, live, except pure-bred breeding 2014 Export"	298824624.0
"Bovine animals, live, except pure-bred breeding 2009 Export"	257772216.0
"Bovine animals, live, except pure-bred breeding 2012 Export"	242159350.0
"Bovine animals, live, except pure-bred breeding 2007 Export"	199888037.0
"Bovine animals, live, except pure-bred breeding 2011 Export"	194178940.0
"Bovine animals, live, except pure-bred breeding 2008 Export"	193144870.0
"Bovine animals, live, except pure-bred breeding 1995 Import"	126427408.0
"Bovine animals, liv

Teste com base completa

In [199]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224033.407701
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224033.407701...

Top 10 Entradas da atividade
"Iron ore, concentrate, not iron pyrites,unagglomerate 2016 Export"	344548049536.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2015 Export"	315020626912.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2014 Export"	294462274531.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2013 Export"	282152705950.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2012 Export"	275398874690.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2011 Export"	274796905500.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2010 Export"	258820300767.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2009 Export"	235775496309.0
"Iron ore, concentrate

### Informação 12
 - Preco medio das commodities de acordo com o ano

In [200]:
%%file pratica.py
from mrjob.job import MRJob
import sys
# Médio do preço considerada, no próximo exercício realizo a mediana.
class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        comm = campos[3]
        price = float(campos[5])
        yield(comm + " " + year, price)
      except:
        pass

    def reducer(self, chave, valores):
      count = 0
      valor = 0
      for v in valores:
        valor = valor + v
        count += 1
      avg = valor / count
      yield(chave, avg)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [201]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224043.372243
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224043.372243...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding 2016"	87946336.62244898
"Bovine animals, live, except pure-bred breeding 2014"	87511974.93283582
"Bovine animals, live, except pure-bred breeding 1994"	84802246.29166667
"Bovine animals, live, except pure-bred breeding 2013"	83745374.140625
"Bovine animals, live, except pure-bred breeding 2010"	82026854.73728813
"Bovine animals, live, except pure-bred breeding 2015"	80993941.36
"Bovine animals, live, except pure-bred breeding 2012"	80702915.28148147
"Bovine animals, live, except pure-bred breeding 2011"	79034358.08333333
"Bovine animals, live, except pure-bred breeding 1995"	77522772.4090909
"Swine, live except pure-bred breeding

Teste com base completa

In [202]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224044.412058
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224044.412058...

Top 10 Entradas da atividade
"Petroleum oils, oils from bituminous minerals, crude 2013"	13723863576.050724
"Petroleum oils, oils from bituminous minerals, crude 2012"	13609996020.539568
"Petroleum oils, oils from bituminous minerals, crude 2011"	13472714007.958334
"Petroleum oils, oils from bituminous minerals, crude 2014"	13316552152.374046
"Petroleum oils, oils from bituminous minerals, crude 2008"	11022548229.34694
"Petroleum oils, oils from bituminous minerals, crude 2010"	10657872601.458647
"Petroleum oils, oils from bituminous minerals, crude 2007"	7958776169.427587
"Petroleum oils, oils from bituminous minerals, crude 2006"	7945934464.775194
"Petroleum oils, oils from bituminous minerals, crude 200

### Informação 13
 - Valores maximos, minimos e médios de cada tipo de mercadoria por ano

In [203]:
%%file pratica.py
from mrjob.job import MRJob
import sys
import statistics as stat

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        comm = campos[3]
        price = float(campos[5])
        # Caso queira considerar o valor individual da mercadoria em questão, ao invés do valor total, se deve usar as variáveis declaradas e comentadas abaixo.
        # qnt = float(campos[8])
        # adjustedIndividualPrice = price / qnt
        yield(comm + " " + year, price)
      except:
        pass

    def reducer(self, chave, valores):
      values = list(valores)
      maxVal = max(values)
      medianVal = stat.median(values)
      minVal = min(values)
      yield(chave, "MAX: " + str(maxVal) + " MEDIAN: " + str(medianVal) + " MIN: " + str(minVal))

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [204]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224134.439847
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224134.439847...

Top 10 Entradas da atividade
"Animals, live, except farm animals 1988"	"MAX: 18051537.0 MEDIAN: 904732.0 MIN: 2189.0"
"Animals, live, except farm animals 1989"	"MAX: 19134896.0 MEDIAN: 919690.0 MIN: 8860.0"
"Animals, live, except farm animals 1990"	"MAX: 27268272.0 MEDIAN: 809465.0 MIN: 89.0"
"Animals, live, except farm animals 1991"	"MAX: 26308977.0 MEDIAN: 361021.0 MIN: 85.0"
"Animals, live, except farm animals 1992"	"MAX: 35839220.0 MEDIAN: 363299.0 MIN: 50.0"
"Animals, live, except farm animals 1993"	"MAX: 28373560.0 MEDIAN: 215039.0 MIN: 12.0"
"Animals, live, except farm animals 1994"	"MAX: 37554432.0 MEDIAN: 254861.5 MIN: 111.0"
"Animals, live, except farm animals 1995"	"MAX: 39840416.0 MEDIAN: 18469

Teste com base completa

In [205]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224135.566442
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224135.566442...

Top 10 Entradas da atividade
"1,2,3,4,5,6-hexachlorocyclohexane 1988"	"MAX: 1649.0 MEDIAN: 1649.0 MIN: 1649.0"
"1,2,3,4,5,6-hexachlorocyclohexane 1989"	"MAX: 13100.0 MEDIAN: 6553.0 MIN: 6.0"
"1,2,3,4,5,6-hexachlorocyclohexane 1990"	"MAX: 18254.0 MEDIAN: 18254.0 MIN: 18254.0"
"1,2,3,4,5,6-hexachlorocyclohexane 1991"	"MAX: 7848.0 MEDIAN: 7848.0 MIN: 7848.0"
"1,2,3,4,5,6-hexachlorocyclohexane 1992"	"MAX: 25588.0 MEDIAN: 25588.0 MIN: 25588.0"
"1,2,3,4,5,6-hexachlorocyclohexane 1993"	"MAX: 258476.0 MEDIAN: 140132.5 MIN: 21789.0"
"1,2,3,4,5,6-hexachlorocyclohexane 1994"	"MAX: 139529.0 MEDIAN: 105506.0 MIN: 4553.0"
"1,2,3,4,5,6-hexachlorocyclohexane 1995"	"MAX: 484879.0 MEDIAN: 74716.0 MIN: 42.0"
"1,2,3,4,5,6-hex

### Informação 14
 - Pais com o maior preco da commodity em fluxos do tipo Export

In [206]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        country = campos[0]
        comm = campos[3]
        flow = campos[4]
        price = float(campos[5])
        if flow == "Export":
          yield(country + " - " + comm , price)
      except:
        pass

    def reducer(self, chave, valores):
      values = list(valores)
      maxVal = max(values)
      yield(chave, maxVal)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [207]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224222.236400
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224222.236400...

Top 10 Entradas da atividade
"France - Bovine animals, live, except pure-bred breeding"	1895889152.0
"Canada - Bovine animals, live, except pure-bred breeding"	1798687283.0
"Australia - Bovine animals, live, except pure-bred breeding"	947455270.0
"Netherlands - Swine, live except pure-bred breeding > 50 kg"	902682671.0
"Denmark - Swine, live except pure-bred breeding < 50 kg"	881569879.0
"Mexico - Bovine animals, live, except pure-bred breeding"	870822705.0
"Brazil - Bovine animals, live, except pure-bred breeding"	718200007.0
"EU-28 - Bovine animals, live, except pure-bred breeding"	711362708.0
"Netherlands - Swine, live except pure-bred breeding < 50 kg"	698861013.0
"Germany - Fowls, live domestic > 185 

Teste com base completa

In [208]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224223.171809
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224223.171809...

Top 10 Entradas da atividade
"EU-28 - Oils petroleum, bituminous, distillates, except crude"	130265972772.0
"Iraq - Petroleum oils, oils from bituminous minerals, crude"	94027500000.0
"Canada - Petroleum oils, oils from bituminous minerals, crude"	88119818380.0
"Iran - Petroleum oils, oils from bituminous minerals, crude"	84381572020.0
"Kuwait - Petroleum oils, oils from bituminous minerals, crude"	79041032289.0
"EU-28 - Medicaments nes, in dosage"	77054946304.0
"Angola - Petroleum oils, oils from bituminous minerals, crude"	68863266749.0
"India - Oils petroleum, bituminous, distillates, except crude"	67075185518.0
"Australia - Iron ore, concentrate, not iron pyrites,unagglomerate"	66917183629.0
"Kazakhsta

### Informação 15
 - Quantidade de transações comerciais de acordo com o fluxo, de acordo com o ano

In [209]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        flow = campos[4]
        yield(year + " - " + flow, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [210]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224246.388891
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224246.388891...

Top 10 Entradas da atividade
"2012 - Import"	928
"2013 - Import"	922
"2010 - Import"	899
"2011 - Import"	899
"2007 - Import"	898
"2003 - Import"	893
"2014 - Import"	891
"2000 - Import"	877
"2015 - Import"	861
"2001 - Import"	860


Teste com base completa

In [211]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.224247.581109
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.224247.581109...

Top 10 Entradas da atividade
"2007 - Import"	82032
"2010 - Import"	81859
"2009 - Import"	81463
"2012 - Import"	81362
"2006 - Import"	81274
"2011 - Import"	81225
"2005 - Import"	79987
"2013 - Import"	79329
"2004 - Import"	79280
"2008 - Import"	79214
